In [ ]:
# %reload_ext autoreload
# %autoreload 2
# %matplotlib inline

# from fastai.vision import *
# from fastai.tabular import *
# from image_tabular.core import *
# from image_tabular.dataset import *
# from image_tabular.model import *
# from image_tabular.metric import *
# import torch


# use gpu by default if available


In [ ]:
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:

# growth_rate=32 
# block_config=(2, 2, 2, 2)
# num_init_features=64
# bn_size=4
# drop_rate=0.5
# batch_size=128
# patch_size=256
# gpuid = 0
# num_classes = 4
# random.seed(7654788126099255772) 
# print(torch.cuda.get_device_properties(gpuid))
# torch.cuda.set_device(gpuid)
# device = torch.device(f'cuda:{gpuid}' if torch.cuda.is_available() else 'cpu')
# mdl = DenseNet(growth_rate=growth_rate, block_config=block_config,
#                  num_init_features=num_init_features, 
#                  bn_size=bn_size, 
#                  drop_rate=drop_rate, 
#                  num_classes=num_classes).to(device)

In [ ]:
# dataname="CUP010722"
# checkpoint = torch.load(f"{dataname}_densenet_best_model.pth")
# mdl.load_state_dict(checkpoint["model_dict"])



In [ ]:
# list(mdl.children())

In [ ]:
# mdl.classifier

In [ ]:
# mdl.classifier = torch.nn.Sequential(*list(mdl.classifier.children())[:-1])
# mdl

In [ ]:
# import torch.nn as nn
# import torch.nn.functional as F

# # increasing few layers in our model
# class DenseNetFusion(nn.Module):
#     def __init__(self):
#         super(DenseNetFusion, self).__init__()
#         self.model = mdl 
#         self.image_dense_layer_1 = nn.Linear(128 , 64)
#         self.relu = nn.ReLU()
#         self.batchnorm = nn.BatchNorm1d(64)
#         self.dropout = nn.Dropout2d(0.5)
#         self.image_dense_layer_2 = nn.Linear(64, 10)
#         self.BilinearPool = nn.Bilinear(10, 10, 10)
#         self.final = nn.Linear(10 , 4)
# #         self.tabular_dense_layer_1 = nn.Linear(10, 8)
# #         self.tabular_dense_layer_2 = nn.Linear(8, 5)        
        
    
        
        
#     def forward(self, image_inputs , tabular_data_inputs):
#         x_img = self.model.eval(image_inputs)
#         # Pooling and final linear layer
#         x_img = x_img.flatten(start_dim=1)
#         x_img = self.image_dense_layer_1(x_img)
#         x_img = self.relu(x_img)
#         x_img = self.batchnorm(x_img)
#         x_img = self.dropout(x_img)
#         x_img = self.image_dense_layer_2(x_img)
#         x_img = self.relu(x_img)
        
        
#         tab = self.tabular_dense_layer_1(tabular_data_inputs)
#         tab = self.relu(tab)
#         tab = self.tabular_dense_layer_2(tab)
#         tab = self.relu(tab)
#         x = self.BilinearPool(x_img,tab)
# #         x = torch.cat((x, tab), dim=1)

#         return self.final(x)

In [ ]:
# # m = nn.ReLU()
# model = DenseNetFusion()
# model = model.to(device)
# model

In [ ]:
# m = nn.Bilinear(5, 5, 5)
# input1 = torch.randn(1, 5)
# input2 = torch.randn(1, 5)
# output = m(input1, input2)
# print(output.size())

In [1]:
#v3.classification
#28/11/2018

dataname="CUPBinlinFusion"
gpuid=0

# --- densenet params
#these parameters get fed directly into the densenet class, and more description of them can be discovered there
num_classes=4    #number of classes in the data mask that we'll aim to predict
in_channels= 3  #input channel of the data, RGB = 3


growth_rate=32 #down from 32 
block_config=(2, 2, 2, 2)
num_init_features=64
bn_size=4
drop_rate=0.5



# --- training params
batch_size=128
patch_size=256 #currently, this needs to be 224 due to densenet architecture
num_epochs = 100
phases = ["train","val"] #how many phases did we create databases for?
validation_phases= ["val"] #when should we do valiation? note that validation is *very* time consuming, so as opposed to doing for both training and validation, we do it only for vlaidation at the end of the epoch
                           #additionally, using simply [], will skip validation entirely, drastically speeding things up

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import DenseNet

import PIL
import matplotlib.pyplot as plt
import cv2

import numpy as np
import sys, glob

from tensorboardX import SummaryWriter

import time
import math
import tables

import random

from sklearn.metrics import confusion_matrix

In [3]:
#helper function for pretty printing of current time and remaining time
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)
def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent+.00001)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [4]:
#specify if we should use a GPU (cuda) or only the CPU
print(torch.cuda.get_device_properties(gpuid))
torch.cuda.set_device(gpuid)
device = torch.device(f'cuda:{gpuid}' if torch.cuda.is_available() else 'cpu')

_CudaDeviceProperties(name='NVIDIA GeForce RTX 2080 Ti', major=7, minor=5, total_memory=11263MB, multi_processor_count=68)


In [5]:
#build the model according to the paramters specified above and copy it to the GPU. finally print out the number of trainable parameters
 
# model = DenseNet(growth_rate=growth_rate, block_config=block_config,
#                  num_init_features=num_init_features, 
#                  bn_size=bn_size, 
#                  drop_rate=drop_rate, 
#                  num_classes=num_classes).to(device)
# #model = DenseNet(growth_rate=32, block_config=(6, 12, 24, 16), #these represent the default parameters
# #                 num_init_features=64, bn_size=4, drop_rate=0, num_classes=3)

# print(f"total params: \t{sum([np.prod(p.size()) for p in model.parameters()])}")

In [6]:
import torch
from torchvision.models import DenseNet
import numpy as np
import os
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import tables
import random
import cv2
import glob
import random
import pandas as pd
import re

growth_rate=32 
block_config=(2, 2, 2, 2)
num_init_features=64
bn_size=4
drop_rate=0.5
batch_size=128
patch_size=256
gpuid = 0
num_classes = 4
random.seed(7654788126099255772) 
print(torch.cuda.get_device_properties(gpuid))
torch.cuda.set_device(gpuid)
device = torch.device(f'cuda:{gpuid}' if torch.cuda.is_available() else 'cpu')
# mdl = DenseNet(growth_rate=growth_rate, block_config=block_config,
#                  num_init_features=num_init_features, 
#                  bn_size=bn_size, 
#                  drop_rate=drop_rate, 
#                  num_classes=num_classes).to(device)
# pretrained_dataname="CUP010722"
# checkpoint = torch.load(f"{pretrained_dataname}_densenet_best_model.pth")
# mdl.load_state_dict(checkpoint["model_dict"])
# mdl.classifier = torch.nn.Sequential(*list(mdl.classifier.children())[:-1])


_CudaDeviceProperties(name='NVIDIA GeForce RTX 2080 Ti', major=7, minor=5, total_memory=11263MB, multi_processor_count=68)


In [7]:
import torch.nn as nn
import torch.nn.functional as F

# increasing few layers in our model
class DenseNetFusion(nn.Module):
    def __init__(self):
        super(DenseNetFusion, self).__init__()
        self.image_dense_layer_1 = nn.Linear(300 , 100)
        self.relu = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm1d(100)
        self.dropout = nn.Dropout2d(0.5)
        self.image_dense_layer_2 = nn.Linear(100, 10)

        
        
        self.BilinearPool = nn.Bilinear(10, 10, 10)
        self.final = nn.Linear(10 , 4)
        self.tabular_dense_layer_1 = nn.Linear(10, 10)
#         self.tabular_dense_layer_2 = nn.Linear(8, 5)        
        
    
        
        
    def forward(self, image_inputs , tabular_data_inputs):
#       DL data
        x_img = image_inputs
        x_img = self.image_dense_layer_1(x_img)
        x_img = self.relu(x_img)
        x_img = self.batchnorm1(x_img)
        x_img = self.dropout(x_img)
        x_img = self.image_dense_layer_2(x_img)
        x_img = self.relu(x_img)

        
#       Tabular data
        tab = self.tabular_dense_layer_1(torch.tensor(tabular_data_inputs, device=device).float())
        tab = self.relu(tab)
        x = self.BilinearPool(x_img,tab)
#         x = torch.cat((x, tab), dim=1)

        return self.final(x)

In [33]:
import torch.nn as nn
import torch.nn.functional as F

# increasing few layers in our model
class DenseNetFusion(nn.Module):
    def __init__(self):
        super(DenseNetFusion, self).__init__()
        self.image_dense_layer_1 = nn.Linear(300 , 100)
        self.relu = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm1d(100)
        self.dropout = nn.Dropout2d(0.5)
        self.image_dense_layer_2 = nn.Linear(100, 10)

        
        
        self.BilinearPool = nn.Bilinear(10, 10, 10)
        self.final = nn.Linear(20 , 4)
        self.tabular_dense_layer_1 = nn.Linear(10, 10)
#         self.tabular_dense_layer_2 = nn.Linear(8, 5)        
        
    
        
        
    def forward(self, image_inputs , tabular_data_inputs):
#       DL data
        x_img = image_inputs
        x_img = self.image_dense_layer_1(x_img)
        x_img = self.relu(x_img)
        x_img = self.batchnorm1(x_img)
        x_img = self.dropout(x_img)
        x_img = self.image_dense_layer_2(x_img)
        x_img = self.relu(x_img)

        
#       Tabular data
        tab = self.tabular_dense_layer_1(torch.tensor(tabular_data_inputs, device=device).float())
        tab = self.relu(tab)
        x = torch.cat((x_img,tab), dim=1)
#         x = torch.cat((x, tab), dim=1)

        return self.final(x)

In [34]:
model = DenseNetFusion()
model = model.to(device)
model

DenseNetFusion(
  (image_dense_layer_1): Linear(in_features=300, out_features=100, bias=True)
  (relu): ReLU()
  (batchnorm1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout2d(p=0.5, inplace=False)
  (image_dense_layer_2): Linear(in_features=100, out_features=10, bias=True)
  (BilinearPool): Bilinear(in1_features=10, in2_features=10, out_features=10, bias=True)
  (final): Linear(in_features=20, out_features=4, bias=True)
  (tabular_dense_layer_1): Linear(in_features=10, out_features=10, bias=True)
)

In [35]:
params = list(model.image_dense_layer_1.parameters()) + \
         list(model.image_dense_layer_2.parameters()) + \
         list(model.tabular_dense_layer_1.parameters()) + \
         list(model.final.parameters())

In [36]:
print(f"total params: \t{sum([np.prod(p.size()) for p in params])}")

total params: 	31304


In [37]:
#this defines our dataset class which will be used by the dataloader
class Dataset(object):
    def __init__(self, fname ,img_transform=None):
        #nothing special here, just internalizing the constructor parameters
        self.fname=fname

        self.img_transform=img_transform
        
        with tables.open_file(self.fname,'r') as db:
            self.classsizes=db.root.classsizes[:]
            self.nitems=db.root.imgs.shape[0]
        
        self.imgs = None
        self.labels = None
        self.hc = None
        
    def __getitem__(self, index):
        #opening should be done in __init__ but seems to be
        #an issue with multithreading so doing here. need to do it everytime, otherwise hdf5 crashes

        with tables.open_file(self.fname,'r') as db:
            self.imgs=db.root.DLfeats
            self.labels=db.root.labels
            self.hc = db.root.HCFeats

            #get the requested image and mask from the pytable
            img = self.imgs[index,:]
            label = self.labels[index]
            handFeats = self.hc[index,:]
        
        
        img_new = img
        
        if self.img_transform is not None:
            img_new = self.img_transform(img)


        return img_new, label, img, handFeats
    def __len__(self):
        return self.nitems

In [38]:
img_transform = transforms.Compose([
     transforms.ToPILImage(),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(size=(patch_size,patch_size),pad_if_needed=True), 
    transforms.RandomResizedCrop(size=patch_size),
    transforms.RandomRotation(180),
    transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=.5),
    transforms.RandomGrayscale(),
    transforms.ToTensor()
    ])


dataset={}
dataLoader={}
for phase in phases: #now for each of the phases, we're creating the dataloader
                     #interestingly, given the batch size, i've not seen any improvements from using a num_workers>0
    
    dataset[phase]=Dataset(f"./{dataname}_{phase}.pytable", img_transform=None)
    dataLoader[phase]=DataLoader(dataset[phase], batch_size=batch_size, 
                                shuffle=True, num_workers=0,pin_memory=True) #previously num_workers=8
    print(f"{phase} dataset size:\t{len(dataset[phase])}")


train dataset size:	15870
val dataset size:	6802


In [39]:
dataset["train"]

In [40]:
#visualize a single example to verify that it is correct
(img, label, img_old, handFeats)=dataset["train"][27]
# fig, ax = plt.subplots(1,2, figsize=(10,4))  # 1 row, 2 columns
handFeats
#build output showing patch after augmentation and original patch
# ax[0].imshow(np.moveaxis(img,0,-1))
# ax[1].imshow(img_old)

print(label)
img.shape

1


(300,)

In [45]:
optim = torch.optim.Adam(params,lr=1e-5) #adam is going to be the most robust, though perhaps not the best performing, typically a good place to start
# optim = torch.optim.SGD(model.parameters(),
#                           lr=.1,
#                           momentum=0.9,
#                           weight_decay=0.0005)

In [46]:
optim

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 1e-05
    weight_decay: 0
)

In [47]:
#we have the ability to weight individual classes, in this case we'll do so based on their presense in the trainingset
#to avoid biasing any particular class
nclasses = dataset["train"].classsizes.shape[0]
class_weight=dataset["train"].classsizes
class_weight = torch.from_numpy(1-class_weight/class_weight.sum()).type('torch.FloatTensor').to(device)

print(class_weight) #show final used weights, make sure that they're reasonable before continouing
criterion = nn.CrossEntropyLoss(weight = class_weight) 

tensor([0.6786, 0.7062, 0.7739, 0.8413], device='cuda:0')


In [48]:
#def trainnetwork():
num_epochs = 300
writer=SummaryWriter() #open the tensorboard visualiser
best_loss_on_test = np.Infinity

start_time = time.time()
for epoch in range(num_epochs):
    #zero out epoch based performance variables 
    all_acc = {key: 0 for key in phases} 
    all_loss = {key: torch.zeros(0).to(device) for key in phases} #keep this on GPU for greatly improved performance
    cmatrix = {key: np.zeros((num_classes,num_classes)) for key in phases}

    for phase in phases: #iterate through both training and validation states

        if phase == 'train':
            model.train()  # Set model to training mode
        else: #when in eval mode, we don't want parameters to be updated
            model.eval()   # Set model to evaluate mode

        for ii , (X, label, img_orig,tab) in enumerate(dataLoader[phase]): #for each of the batches
            X = X.to(device)  # [Nbatch, 3, H, W]
            if np.any(np.isnan(X.cpu().numpy())) or np.any(np.isnan(tab.cpu().numpy())):
                breakpoint()
            label = label.type('torch.LongTensor').to(device)  # [Nbatch, 1] with class indices (0, 1, 2,...num_classes)

            with torch.set_grad_enabled(phase == 'train'): #dynamically set gradient computation, in case of validation, this isn't needed
                                                            #disabling is good practice and improves inference time

                prediction = model(X,tab)  # [N, Nclass]
                loss = criterion(prediction, label)


                if phase=="train": #in case we're in train mode, need to do back propogation
                    optim.zero_grad()
                    loss.backward()
                    optim.step()
                    train_loss = loss


                all_loss[phase]=torch.cat((all_loss[phase],loss.detach().view(1,-1)))

                if phase in validation_phases: #if this phase is part of validation, compute confusion matrix
                    p=prediction.detach().cpu().numpy()
                    cpredflat=np.argmax(p,axis=1).flatten()
                    yflat=label.cpu().numpy().flatten()

                    cmatrix[phase]=cmatrix[phase]+confusion_matrix(yflat,cpredflat, labels=range(nclasses))

        all_acc[phase]=(cmatrix[phase]/cmatrix[phase].sum()).trace()
        all_loss[phase] = all_loss[phase].cpu().numpy().mean()

        #save metrics to tensorboard
        writer.add_scalar(f'{phase}/loss', all_loss[phase], epoch)
        if phase in validation_phases:
            writer.add_scalar(f'{phase}/acc', all_acc[phase], epoch)
            for r in range(nclasses):
                for c in range(nclasses): #essentially write out confusion matrix
                    writer.add_scalar(f'{phase}/{r}{c}', cmatrix[phase][r][c],epoch)

    print('%s ([%d/%d] %d%%), train loss: %.4f test loss: %.4f' % (timeSince(start_time, (epoch+1) / num_epochs), 
                                                 epoch+1, num_epochs ,(epoch+1) / num_epochs * 100, all_loss["train"], all_loss["val"]),end="")    

    #if current loss is the best we've seen, save model state with all variables
    #necessary for recreation
    if all_loss["val"] < best_loss_on_test:
        best_loss_on_test = all_loss["val"]
        print("  **")
        state = {'epoch': epoch + 1,
         'model_dict': model.state_dict(),
         'optim_dict': optim.state_dict(),
         'best_loss_on_test': all_loss,
         'in_channels': in_channels,
         'growth_rate':growth_rate,
         'block_config':block_config,
         'num_init_features':num_init_features,
         'bn_size':bn_size,
         'drop_rate':drop_rate,
         'num_classes':num_classes}


        torch.save(state, f"{dataname}_densenet_best_model.pth")
    else:
        print("")

e:\mvp\venv\lib\site-packages\ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
e:\mvp\venv\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in true_divide


0m 29s (- 72m 56s) ([1/150] 0%), train loss: 2.3630 test loss: 7.9739  **
0m 58s (- 71m 46s) ([2/150] 1%), train loss: 2.3219 test loss: 7.8271  **
1m 27s (- 71m 1s) ([3/150] 2%), train loss: 2.2932 test loss: 7.8819
1m 55s (- 70m 15s) ([4/150] 2%), train loss: 2.2602 test loss: 7.8776
2m 23s (- 69m 33s) ([5/150] 3%), train loss: 2.2342 test loss: 7.6950  **
2m 52s (- 68m 59s) ([6/150] 4%), train loss: 2.2112 test loss: 7.6345  **
3m 20s (- 68m 23s) ([7/150] 4%), train loss: 2.1894 test loss: 7.7193
3m 49s (- 67m 56s) ([8/150] 5%), train loss: 2.1736 test loss: 7.5850  **
4m 18s (- 67m 28s) ([9/150] 6%), train loss: 2.1566 test loss: 7.5111  **
4m 47s (- 66m 58s) ([10/150] 6%), train loss: 2.1449 test loss: 7.4790  **
5m 15s (- 66m 31s) ([11/150] 7%), train loss: 2.1313 test loss: 7.5285
5m 44s (- 66m 1s) ([12/150] 8%), train loss: 2.1176 test loss: 7.4409  **
6m 13s (- 65m 33s) ([13/150] 8%), train loss: 2.1116 test loss: 7.4140  **
6m 41s (- 65m 3s) ([14/150] 9%), train loss: 2.0989 

52m 52s (- 18m 34s) ([111/150] 74%), train loss: 1.5224 test loss: 5.1048  **
53m 21s (- 18m 6s) ([112/150] 74%), train loss: 1.5186 test loss: 5.0710  **
53m 49s (- 17m 37s) ([113/150] 75%), train loss: 1.5181 test loss: 5.1119
54m 18s (- 17m 8s) ([114/150] 76%), train loss: 1.5144 test loss: 5.0797
54m 46s (- 16m 40s) ([115/150] 76%), train loss: 1.5106 test loss: 5.0644  **
55m 15s (- 16m 11s) ([116/150] 77%), train loss: 1.5105 test loss: 5.0712
55m 43s (- 15m 42s) ([117/150] 78%), train loss: 1.5067 test loss: 5.0530  **
56m 12s (- 15m 14s) ([118/150] 78%), train loss: 1.5073 test loss: 5.0604
56m 40s (- 14m 45s) ([119/150] 79%), train loss: 1.5026 test loss: 5.0306  **
57m 9s (- 14m 17s) ([120/150] 80%), train loss: 1.5032 test loss: 5.0355
57m 37s (- 13m 48s) ([121/150] 80%), train loss: 1.5005 test loss: 5.0034  **
58m 6s (- 13m 20s) ([122/150] 81%), train loss: 1.4971 test loss: 5.1054
58m 34s (- 12m 51s) ([123/150] 82%), train loss: 1.4952 test loss: 4.9914  **
59m 3s (- 12m 

In [ ]:
X.cpu().numpy()

In [ ]:
np.any(np.isnan(tab))

In [ ]:
tab.cpu()